##### Create a JSON API (REST or GraphQL) (using any language + framework you're comfortable with) which will return which cryptocurrency exchange we should use to buy a given amount of Bitcoin to minimise the amount of USD or USDT we'll spend on this trade.

In [40]:
import requests
import numpy as np

In [144]:
response = requests.get("https://api.binance.com/api/v3/depth?symbol=BTCUSDT&limit=5000")
binance_ask = np.array(response.json()["asks"],dtype = float)
binance_ask

array([[1.942077e+04, 4.000000e-02],
       [1.942083e+04, 8.715400e-01],
       [1.942084e+04, 1.121840e+00],
       ...,
       [1.984033e+04, 5.040000e-03],
       [1.984035e+04, 5.100000e-04],
       [1.984036e+04, 9.829000e-02]])

In [145]:
response = requests.get(url = "https://api.exchange.coinbase.com/products/BTC-USDT/book?level=2", headers={"accept": "application/json"})
coinbase_ask = np.array(response.json()["asks"],dtype = float)
coinbase_ask

array([[1.94215600e+04, 2.00000000e-03, 1.00000000e+00],
       [1.94215700e+04, 3.85000000e-02, 1.00000000e+00],
       [1.94220900e+04, 1.20000000e-01, 1.00000000e+00],
       ...,
       [2.67928822e+06, 4.93290000e-04, 1.00000000e+00],
       [2.93000000e+06, 1.81085700e-02, 1.00000000e+00],
       [1.00000000e+10, 1.00000000e-02, 1.00000000e+00]])

In [146]:
response = requests.get(" https://ftx.com/api/markets/BTC/USDT/orderbook?depth=100")
ftx_ask = np.array(response.json()["result"]["asks"],dtype = float)
ftx_ask

array([[1.94180e+04, 1.95780e+00],
       [1.94190e+04, 1.84600e+00],
       [1.94200e+04, 3.63760e+00],
       [1.94210e+04, 2.32130e+00],
       [1.94220e+04, 2.59930e+00],
       [1.94230e+04, 2.74140e+00],
       [1.94240e+04, 2.81600e-01],
       [1.94250e+04, 2.60270e+00],
       [1.94260e+04, 2.32200e-01],
       [1.94270e+04, 1.41350e+00],
       [1.94280e+04, 1.00000e-01],
       [1.94290e+04, 1.02000e-02],
       [1.94310e+04, 3.87000e-02],
       [1.94320e+04, 1.62300e-01],
       [1.94330e+04, 2.12800e-01],
       [1.94350e+04, 2.13300e-01],
       [1.94360e+04, 4.12500e-01],
       [1.94370e+04, 4.31500e-01],
       [1.94380e+04, 1.04310e+00],
       [1.94390e+04, 9.71500e-01],
       [1.94400e+04, 2.33800e-01],
       [1.94410e+04, 1.72420e+00],
       [1.94420e+04, 3.83370e+00],
       [1.94430e+04, 3.93440e+00],
       [1.94440e+04, 7.90200e+00],
       [1.94450e+04, 7.63900e-01],
       [1.94460e+04, 2.59140e+00],
       [1.94470e+04, 2.02850e+00],
       [1.94490e+04,

In [135]:
len(binance_ask), len(coinbase_ask), len(ftx_ask)

(5000, 4127, 100)

In [209]:
amount = 1.2
def get_best_price(amount):
    btcAmount = None
    usdAmount = None
    Exchange = None
    
    response = requests.get("https://api.binance.com/api/v3/depth?symbol=BTCUSDT&limit=5000")
    binance_ask = np.array(response.json()["asks"],dtype = float)
    
    response = requests.get(url = "https://api.exchange.coinbase.com/products/BTC-USDT/book?level=2", headers={"accept": "application/json"})
    coinbase_ask = np.array(response.json()["asks"],dtype = float)
    
    response = requests.get(" https://ftx.com/api/markets/BTC/USDT/orderbook?depth=100")
    ftx_ask = np.array(response.json()["result"]["asks"],dtype = float)
    
    order_by = 1
    
    filtered_binance = binance_ask[binance_ask[:,1] >= amount ]
    filtered_binance = filtered_binance[filtered_binance[:, order_by].argsort()]
    
    filtered_coinbase = coinbase_ask[coinbase_ask[:,1] >= amount ]
    filtered_coinbase = filtered_coinbase[filtered_coinbase[:, order_by].argsort()]
    
    filtered_ftx = ftx_ask[ftx_ask[:,1] >= amount ]
    filtered_ftx = filtered_ftx[filtered_ftx[:, order_by].argsort()]
    
    exchanges = ["Binance", "Coinbase", "FTX"]
    filtered_exchange_order_books = [filtered_binance, filtered_coinbase, filtered_ftx]
    
    lowest_asks = []
    for filtered in filtered_exchange_order_books:
        if len(filtered) != 0:
            lowest_asks.append(filtered[0,0])
        else:
            lowest_asks.append(float('inf'))
            
    min_index = lowest_asks.index(min(lowest_asks))
    try:
        btcAmount = filtered_exchange_order_books[min_index][0,1]
        usdAmount = filtered_exchange_order_books[min_index][0,0]
        Exchange = exchanges[min_index]
    except:
        pass
    
    return {"btcAmount": btcAmount,"usdAmount": usdAmount ,"Exchange":Exchange}

get_best_price(amount)

{'btcAmount': 1.4403, 'usdAmount': 19352.0, 'Exchange': 'FTX'}

In [156]:
filtered_binance = binance_ask[binance_ask[:,1] >= amount ]
filtered_binance = filtered_binance[filtered_binance[:, 1].argsort()]
binance_lowest_ask = filtered_binance[0,0]
binance_lowest_ask
# filtered

19510.0

In [170]:
filtered_coinbase = coinbase_ask[coinbase_ask[:,1] >= amount ]
filtered_coinbase = filtered_coinbase[filtered_coinbase[:, 1].argsort()]
coinbase_lowest_ask = filtered_coinbase[0,0]
coinbase_lowest_ask

IndexError: index 0 is out of bounds for axis 0 with size 0

In [158]:
filtered_ftx = ftx_ask[ftx_ask[:,1] >= amount ]
filtered_ftx = filtered_ftx[filtered_ftx[:, 1].argsort()]
ftx_lowest_ask = filtered_ftx[0,0]
ftx_lowest_ask

19506.0

In [180]:
float('inf')

inf